In [1]:
import json, os
from azureml.core import Workspace
import base64, io
from PIL import Image
import numpy as np
import azure.cognitiveservices.speech as speechsdk
from utils import authenticate_client, entity_linking, key_phrase_extraction, draw_image

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (cryptography 2.9.2 (c:\users\louisli\miniconda3\lib\site-packages), Requirement.parse('cryptography>=3.3.1; extra == "crypto"'), {'PyJWT'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (msal-extensions 1.0.0 (c:\users\louisli\miniconda3\lib\site-packages), Requirement.parse('msal-extensions~=0.3.0'), {'azure-identity'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (cryptography 2.9.2 (c:\users\louisli\miniconda3\lib\site-packages), Requirement.parse('cryptography>=3.3.1; extra == "crypto"'), {'PyJWT'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pip

In [2]:
#performs one-shot speech recognition from the default microphone
# <SpeechRecognitionWithMicrophone>
speech_config = speechsdk.SpeechConfig(subscription=os.environ["SPEECH_API_KEY"], region="eastus")
# The default language is "en-us".
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

#Steup client for Text Analytics
client = authenticate_client(endpoint = os.environ["COGSVC_ENDPOINT_MP"], 
                                key = os.environ["COGSVC_API_KEY_MP"]
                            )   

while True:
    print("0 - Exit")
    print("1 - Test")

    num = int(input())
    if num==0:
        break
    print("Say something...")
    result = speech_recognizer.recognize_once()

    # Check the result
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(result.text))
        key_phrase_extraction(client, result.text)
        entity_linking(client, result.text)
        draw_image(result.text)
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized")
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
        

0 - Exit
Any number else to test
Say something...
Recognized: Sunset at the lakes of Muskoka.
	Key Phrases:
		 Sunset
		 lakes
		 Muskoka
Linked Entities:

	Name:  Sunset 	Id:  Sunset 	Url:  https://en.wikipedia.org/wiki/Sunset 
	Data Source:  Wikipedia
	Matches:
		Text: Sunset
		Confidence Score: 0.04
		Offset: 0
		Length: 6
	Name:  District Municipality of Muskoka 	Id:  District Municipality of Muskoka 	Url:  https://en.wikipedia.org/wiki/District_Municipality_of_Muskoka 
	Data Source:  Wikipedia
	Matches:
		Text: Muskoka
		Confidence Score: 0.19
		Offset: 23
		Length: 7
Recognized: Sunset at the lakes of Muskoka.


If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


0 - Exit
Any number else to test
